In [1]:
from glob import glob
from tools.procOps import *
from tools.fileOps import *

In [2]:
import os
os.chdir('/public/groups/cgl/cat/primates_evan/')

In [10]:
flnc = glob('isoseq/*/*fasta')

In [5]:
cfg = [x.split() for x in open('assemblies/cactus.config')][2:]
d = {x[0].lower().replace('*', ''): x[1].split('evan/')[1] for x in cfg}

In [35]:
for f in flnc:
    g = f.split('/')[1].split('_')[0]
    a = d[g]
    out = f.replace('.fasta', '.sorted.bam')
    cmd = [['minimap2', '-t', '12', '-ax', 'splice:hq', '-uf', a, f],
            ['samtools', 'view', '-bS', '-'],
            ['sambamba', 'sort', '/dev/stdin', '-o', out]]
    run_proc(cmd)

In [13]:
# run cDNA_cupcake
# first, we need to do BAM -> SAM
for f in flnc:
    bam = f.replace('.fasta', '.sorted.bam')
    sam = f.replace('.fasta', '.sorted.sam')
    !samtools view {bam} > {sam}

In [33]:
from collections import *
grouped_by_genome = defaultdict(list)
for f in flnc:
    g = f.split('/')[1].split('_')[0]
    grouped_by_genome[g].append(f)

In [38]:
bams

'isoseq/bonobo_iPSC/bonobo0_merged_flnc.bam.sorted.bam isoseq/bonobo_iPSC/bonobo1_merged_flnc.bam.sorted.bam isoseq/bonobo_iPSC/bonobo2_merged_flnc.bam.sorted.bam isoseq/bonobo_iPSC/bonobo3_merged_flnc.bam.sorted.bam'

In [55]:
!mkdir -p isoseq/cupcake
import subprocess
# cat everything together to avoid having to merge
for g, l in grouped_by_genome.items():
    out = f'isoseq/cupcake/{g}'
    if len(l) > 1:
        bams = ' '.join([f.replace('.fasta', '.sorted.bam') for f in l])
        !sambamba merge -t 16 {out}.bam {bams}
        !samtools view {out}.bam > {out}.sam
    else:
        !samtools view {l[0].replace('.fasta', '.sorted.bam')} > {out}.sam
    for f in l:
        !cat {f} >> {out}.fa
    subprocess.Popen(['collapse_isoforms_by_sam.py', '--input', f'{out}.fa', '-s', f'{out}.sam', '-o', out])
    print(' '.join(['collapse_isoforms_by_sam.py', '--input', f'{out}.fa', '-s', f'{out}.sam', '-o', out]))

collapse_isoforms_by_sam.py --input isoseq/cupcake/bonobo.fa -s isoseq/cupcake/bonobo.sam -o isoseq/cupcake/bonobo
collapse_isoforms_by_sam.py --input isoseq/cupcake/gibbon.fa -s isoseq/cupcake/gibbon.sam -o isoseq/cupcake/gibbon
collapse_isoforms_by_sam.py --input isoseq/cupcake/gorilla.fa -s isoseq/cupcake/gorilla.sam -o isoseq/cupcake/gorilla
collapse_isoforms_by_sam.py --input isoseq/cupcake/marmoset.fa -s isoseq/cupcake/marmoset.sam -o isoseq/cupcake/marmoset
collapse_isoforms_by_sam.py --input isoseq/cupcake/rhesus.fa -s isoseq/cupcake/rhesus.sam -o isoseq/cupcake/rhesus
collapse_isoforms_by_sam.py --input isoseq/cupcake/human.fa -s isoseq/cupcake/human.sam -o isoseq/cupcake/human
collapse_isoforms_by_sam.py --input isoseq/cupcake/chimp.fa -s isoseq/cupcake/chimp.sam -o isoseq/cupcake/chimp
collapse_isoforms_by_sam.py --input isoseq/cupcake/orangutan.fa -s isoseq/cupcake/orangutan.sam -o isoseq/cupcake/orangutan


In [56]:
%%bash

for f in /public/groups/cgl/cat/primates_evan/isoseq/cupcake/*.collapsed.gff; do
    python /public/home/ifiddes/Comparative-Annotation-Toolkit/programs/convert_cupcake_gff ${f} ${f}3
done

In [ ]:
# repeat for old assemblies
old_fastas = glob('/public/groups/cgl/cat/original_primates/genome_files/*fa')
old_d = {}
for fasta in old_fastas:
    genome = os.path.basename(fasta).split('.')[0].lower()
    if genome in d and genome != 'human':
        old_d[genome] = fasta

In [7]:
out_dir = '/public/groups/cgl/cat/primates_evan/isoseq_original_primates'
for f in flnc:
    g = f.split('/')[1].split('_')[0]
    if g not in old_d:
        continue
    a = old_d[g]
    out = f.replace('.fasta', '.sorted.bam')
    out = out.replace('isoseq', out_dir, 1)
    !mkdir -p {os.path.dirname(out)}
    cmd = [['minimap2', '-t', '12', '-ax', 'splice:hq', '-uf', a, f],
            ['samtools', 'view', '-bS', '-'],
            ['sambamba', 'sort', '/dev/stdin', '-o', out]]
    run_proc(cmd)